In [9]:
import tensorflow as tf
import numpy as np

In [10]:
numFiles = 16
fileNames = []
for file_id in range(numFiles):
    fileNames.append('records_tf_5s/SIGNALS_train_data_strings_' + str(file_id) + '.tfrecord')
dataset = tf.data.TFRecordDataset(fileNames)

In [11]:
def parser(record):
    features = {
                'size': tf.io.FixedLenFeature([], dtype=tf.int64),
                'eeg_1': tf.io.VarLenFeature(dtype=tf.float32),
                'eeg_2': tf.io.VarLenFeature(dtype=tf.float32),
                'eeg_3': tf.io.VarLenFeature(dtype=tf.float32),
                'eeg_4': tf.io.VarLenFeature(dtype=tf.float32),
                'eeg_5': tf.io.VarLenFeature(dtype=tf.float32),
                'eeg_5': tf.io.VarLenFeature(dtype=tf.float32),
                'eeg_6': tf.io.VarLenFeature(dtype=tf.float32),
                'eeg_7': tf.io.VarLenFeature(dtype=tf.float32),
                'pulse': tf.io.VarLenFeature(dtype=tf.float32),
                'x': tf.io.VarLenFeature(dtype=tf.float32),
                'y': tf.io.VarLenFeature(dtype=tf.float32),
                'z': tf.io.VarLenFeature(dtype=tf.float32),
                'label': tf.io.FixedLenFeature([], dtype=tf.int64)

                }
    parsed = tf.io.parse_single_example(record, features)

    to_return = {"eeg_1": parsed["eeg_1"],
            'eeg_2': parsed["eeg_2"],
            'eeg_3': parsed["eeg_3"],
            'eeg_4': parsed["eeg_4"],
            'eeg_5': parsed["eeg_5"],
            'eeg_6': parsed["eeg_6"],
            'eeg_7': parsed["eeg_7"],
            'pulse': parsed["pulse"],
            'x': parsed["x"],
            'y': parsed["y"],
            'z': parsed["z"], 
            "size": parsed["size"]}

    return to_return, parsed["label"]

dataset = dataset.map(parser).batch(1024)
# iterator_dataset = dataset.map(parser).__iter__()

In [12]:
def count(counts, batch):
  features, labels = batch
  class_0 = labels == 0
  class_0 = tf.cast(class_0, tf.int32)

  class_1 = labels == 1
  class_1 = tf.cast(class_1, tf.int32)

  class_2 = labels == 2
  class_2 = tf.cast(class_2, tf.int32)

  class_3 = labels == 3
  class_3 = tf.cast(class_3, tf.int32)

  class_4 = labels == 4
  class_4 = tf.cast(class_4, tf.int32)

  counts['class_0'] += tf.reduce_sum(class_0)
  counts['class_1'] += tf.reduce_sum(class_1)
  counts['class_2'] += tf.reduce_sum(class_2)
  counts['class_3'] += tf.reduce_sum(class_3)
  counts['class_4'] += tf.reduce_sum(class_4)

  return counts

On peut voir que les classes ne sont pas équilibrées

In [29]:
echantillon = dataset.take(3)
counts =echantillon.reduce(
    initial_state={'class_0': 0, 'class_1': 0,'class_2': 0,'class_3': 0,'class_4': 0,},
    reduce_func = count)

counts = np.array([counts['class_0'].numpy(),
                   counts['class_1'].numpy(),
                   counts['class_2'].numpy(),
                   counts['class_3'].numpy(),
                   counts['class_4'].numpy(),
                   ]).astype(np.float32)

fractions = counts/counts.sum()
print(fractions)



[0.15820312 0.04785156 0.36328125 0.26627603 0.16438802]


In [22]:

@tf.autograph.experimental.do_not_convert
def dataset_per_label(dataset):
    dataset_label0 = (
    dataset
        .unbatch()
        .filter(lambda features, label: label==0)
        .repeat())

    dataset_label1 = (
    dataset
        .unbatch()
        .filter(lambda features, label: label==1)
        .repeat())

    dataset_label2 = (
    dataset
        .unbatch()
        .filter(lambda features, label: label==2)
        .repeat())

    dataset_label3 = (
    dataset
        .unbatch()
        .filter(lambda features, label: label==3)
        .repeat())

    dataset_label4 = (
    dataset
        .unbatch()
        .filter(lambda features, label: label==4)
        .repeat())
    return dataset_label0,dataset_label1,dataset_label2,dataset_label3,dataset_label4

dataset_label0,dataset_label1,dataset_label2,dataset_label3,dataset_label4 = dataset_per_label(dataset)


In [30]:
balanced_ds = tf.data.experimental.sample_from_datasets(
    [dataset_label0,dataset_label1,dataset_label2,dataset_label3,dataset_label4], [1/5, 1/5, 1/5, 1/5, 1/5]).batch(1024)


Maintenant les observation piochées sont également distribuées

In [31]:
echantillon = balanced_ds.take(3)
counts =echantillon.reduce(
    initial_state={'class_0': 0, 'class_1': 0,'class_2': 0,'class_3': 0,'class_4': 0,},
    reduce_func = count)

counts = np.array([counts['class_0'].numpy(),
                   counts['class_1'].numpy(),
                   counts['class_2'].numpy(),
                   counts['class_3'].numpy(),
                   counts['class_4'].numpy(),
                   ]).astype(np.float32)

fractions = counts/counts.sum()
print(fractions)


[0.18912761 0.20345052 0.2063802  0.19466145 0.2063802 ]
